# NCBI에서 GenBank파일 불러와 시퀀스 출력

In [4]:
from Bio import Entrez
from Bio import SeqIO

def fetch_record(genbank_id):
    Entrez.email = 'nagabuti13@gmail.com'
    handle = Entrez.efetch(db = 'nucleotide', id = genbank_id, rettype = 'gb', retmode = 'xml')
    records = Entrez.read(handle)
    return records

def get_sequence(records):
    return records[0]['GBSeq_sequence']
    
genbank_id = 'MN908947'
records = fetch_record(genbank_id)
sequence = get_sequence(records)

print(len(sequence))
print(sequence)

29903
...


# 국가 별로 GenBank id로 fasta파일 생성

In [3]:
from Bio import SeqIO
import pandas as pd

def write_sequence(genbank_id, records):
    print("## processing: "+ genbank_id)
    record_id = records[0]["GBSeq_accession-version"]
    record_desc = records[0]["GBSeq_definition"]
    record_seq = records[0]["GBSeq_sequence"]
    with open(genbank_id + ".fasta",'w') as fw:
        fw.write(">" + record_id + " " + record_desc + "\n")
        fw.write(record_seq + "\n")
        
def read_table(file):
    df = pd.read_csv(file, sep="\t")
    return df

file = "COVID-19.ncbi_list.200301.txt"
df = read_table(file)
complete_df = df[df["Gene Region"]=="complete"]
genbank_id_list = complete_df["GenBank"]

for genbank_id in genbank_id_list:
    records = fetch_record(genbank_id)
    write_sequence(genbank_id, records)

## processing: MN908947
## processing: LC522972
## processing: LC522973
## processing: LC522974
## processing: LC522975
## processing: LR757995
## processing: LR757996
## processing: LR757998
## processing: MN938384
## processing: MN975262
## processing: MN985325
## processing: MN988668
## processing: MN988669
## processing: MN988713
## processing: MN994467
## processing: MN994468
## processing: MN996527
## processing: MN996528
## processing: MN996529
## processing: MN996530
## processing: MN996531
## processing: MN997409
## processing: MT007544
## processing: MT019529
## processing: MT019530
## processing: MT019531
## processing: MT019532
## processing: MT019533
## processing: MT020880
## processing: MT020881
## processing: MT027062
## processing: MT027063
## processing: MT027064
## processing: MT039873
## processing: MT039887
## processing: MT039888
## processing: MT039890
## processing: MT044257
## processing: MT044258
## processing: MT049951
## processing: MT066175
## processing: M

# WebLogo 생성

In [13]:
from Bio.Align.Applications import MuscleCommandline

muscle_exe = '/Users/kimjoonsu/muscle3.8.31_i86darwin64'
cmd_line = MuscleCommandline(muscle_exe, input = 'covid.all.fasta',
                             out = 'covid.all.aln', clw = ' ')
print(cmd_line)
stdout, stderr = cmd_line()

/Users/kimjoonsu/muscle3.8.31_i86darwin64 -in covid.all.fasta -out covid.all.aln -clw


In [15]:
seq_dict = {}
with open("covid.all.aln",'r') as fr:
    for line in fr:
        if line.startswith("MUSCLE"):
            continue
        if "*" in line:
            continue
        l = line.strip().split()
        if l:
            genbank_id = l[0]
            seq = l[1]
            if genbank_id not in seq_dict:
                seq_dict[genbank_id] = seq
            else:
                seq_dict[genbank_id] += seq

with open("covid.aln.fasta",'w') as fw:
    for k,v in seq_dict.items():
        fw.write(">"+k+"\n")
        fw.write(v+"\n")

# 계통수 생성

In [1]:
from Bio import Phylo

tree = Phylo.read("covid.all.ph","newick")
Phylo.draw(tree)



FileNotFoundError: [Errno 2] No such file or directory: 'covid.all.ph'